In [44]:
# Import your libraries here
print('Importing librairies...')
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
import requests
import pandas as pd
print('...Done!')

Importing librairies...
...Done!


In [45]:
url = 'https://sofifa.com/players'
r = requests.get(url)
r

<Response [200]>

In [6]:
soup = BeautifulSoup(r.text,'html.parser')

In [49]:
links = []
for i in range(0,20000,60):
    urls = f'https://sofifa.com?offset={i}'
    r = requests.get(urls)
    soup = BeautifulSoup(r.text,'html.parser')

    if i%100 == 0:
        print(f'Offset : {i}')

    for j in range(0,120,2):
        link = soup.select('tr td.col-name')[j].select('a')[0].get('href')
        links.append(link)

Offset : 0
Offset : 300
Offset : 600
Offset : 900
Offset : 1200
Offset : 1500
Offset : 1800
Offset : 2100
Offset : 2400
Offset : 2700
Offset : 3000
Offset : 3300
Offset : 3600
Offset : 3900
Offset : 4200
Offset : 4500
Offset : 4800
Offset : 5100
Offset : 5400
Offset : 5700
Offset : 6000
Offset : 6300
Offset : 6600
Offset : 6900
Offset : 7200
Offset : 7500
Offset : 7800
Offset : 8100
Offset : 8400
Offset : 8700
Offset : 9000
Offset : 9300
Offset : 9600
Offset : 9900
Offset : 10200
Offset : 10500
Offset : 10800
Offset : 11100
Offset : 11400
Offset : 11700
Offset : 12000
Offset : 12300
Offset : 12600
Offset : 12900
Offset : 13200
Offset : 13500
Offset : 13800
Offset : 14100
Offset : 14400
Offset : 14700
Offset : 15000
Offset : 15300
Offset : 15600
Offset : 15900
Offset : 16200
Offset : 16500
Offset : 16800
Offset : 17100
Offset : 17400
Offset : 17700
Offset : 18000
Offset : 18300
Offset : 18600
Offset : 18900
Offset : 19200
Offset : 19500
Offset : 19800
Offset : 20100
Offset : 20400
Offse

In [57]:
player_link = list(set(links))

In [62]:
with open("players_link.text", 'w') as output:
    for row in player_link:
        output.write(str(row) + ',')

In [ ]:
pd.